In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import pandas as pd
import random
from keras.layers import GlobalAveragePooling3D
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Concatenate, BatchNormalization, Activation, Conv3D,MaxPooling3D, Dense, Dropout, Flatten
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,CSVLogger
from keras import regularizers
from keras import losses
from keras import backend as K
from keras.callbacks import History
from keras.layers.merge import Concatenate
import keras
import keras.layers as KL
from keras.layers import Layer
from keras.utils.generic_utils import get_custom_objects
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
## 混和精度 ##
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
#################################################################
with open('ADNI_TRAIN_GMWMCSF_CN_1_600_race.pickle','rb') as f :
    CN = pickle.load(f)
with open('ADNI_TRAIN_GMWMCSF_MCI_1_600_race.pickle', 'rb') as f:
    MCI = pickle.load(f)
with open('ADNI_TRAIN_GMWMCSF_AD_1_600_race.pickle','rb') as f :
    AD = pickle.load(f)
#################################################################
ff = pd.DataFrame(np.concatenate((CN,MCI,AD),axis = 0))

train_npy_input_CN = np.load('ADNI_TRAIN_GMWMCSF_CN_1_600.npy')
train_npy_input_MCI = np.load('ADNI_TRAIN_GMWMCSF_MCI_1_600.npy')
TRAIN_npy_input = np.concatenate((train_npy_input_CN, train_npy_input_MCI),axis = 0)
del train_npy_input_CN,train_npy_input_MCI
train_npy_input_AD = np.load('ADNI_TRAIN_GMWMCSF_AD_1_600.npy')
TRAIN_npy_input = np.concatenate((TRAIN_npy_input, train_npy_input_AD),axis = 0) 
del train_npy_input_AD

ttt = np.zeros([1,11],dtype="float32")
fir = np.concatenate((ttt,ff),axis = 0)
fir = fir[1:,:]

TRAIN_pickle = [] 
for j in fir:
    c = [j[4],j[5]] # age and PTGENDER
    TRAIN_pickle.append(c)

TRAIN_pickle = pd.DataFrame(TRAIN_pickle)
TRAIN_pickle[1].replace("Female", 0).replace("Male", 1)

TRAIN_pickle = np.asarray(TRAIN_pickle).astype('float32')
#正規化
TRAIN_pickle[:,0] = (TRAIN_pickle[:,0]-50)/10   #年齡正規化

fir = pd.DataFrame(fir)
fit_y = []
for i in range(len(fir[7])):
    if (fir[7][i] >= 0.0)&(fir[7][i] <= 0.25):
        fit_y.append([1,0,0,0,0,0])
    elif (fir[7][i] > 0.25)&(fir[7][i] <= 2.75):
        fit_y.append([0,1,0,0,0,0])
    elif (fir[7][i] > 2.75)&(fir[7][i] <= 4.25):
        fit_y.append([0,0,1,0,0,0])
    elif (fir[7][i] > 4.25)&(fir[7][i] <= 9.25):
        fit_y.append([0,0,0,1,0,0])
    elif (fir[7][i] > 9.25)&(fir[7][i] <= 15.75):
        fit_y.append([0,0,0,0,1,0])
    elif fir[7][i] > 15.75:
        fit_y.append([0,0,0,0,0,1])
##############  class_weights  #######################
a = fit_y.count([1,0,0,0,0,0])
b = fit_y.count([0,1,0,0,0,0])
c = fit_y.count([0,0,1,0,0,0])
d = fit_y.count([0,0,0,1,0,0])
e = fit_y.count([0,0,0,0,1,0])
f = fit_y.count([0,0,0,0,0,1])
all_count = a+b+c+d+e+f
sw_dic = [all_count/a ,all_count/b ,all_count/c ,all_count/d ,all_count/e ,all_count/f]
del a,b,c,d,e,f
######################################################
######################################################
######################################################
sw = sw_dic[0]*np.ones(len(fit_y))
for ii in range(len(fit_y)):
    if fit_y[ii] == [1,0,0,0,0,0]:
        sw[ii] = sw_dic[0]
    elif fit_y[ii] == [0,1,0,0,0,0]:
        sw[ii] = sw_dic[1]
    elif fit_y[ii] == [0,0,1,0,0,0]:
        sw[ii] = sw_dic[2]
    elif fit_y[ii] == [0,0,0,1,0,0]:
        sw[ii] = sw_dic[3]
    elif fit_y[ii] == [0,0,0,0,1,0]:
        sw[ii] = sw_dic[4]
    elif fit_y[ii] == [0,0,0,0,0,1]:
        sw[ii] = sw_dic[5]
####################################################
TRAIN_npy_input = np.array(TRAIN_npy_input)
TRAIN_npy_input = np.concatenate((np.expand_dims(TRAIN_npy_input[:,:,:,:,0],axis=-1),np.expand_dims(TRAIN_npy_input[:,:,:,:,2],axis=-1)),axis=-1)
TRAIN_npy_input =TRAIN_npy_input[:, 7:135, 12:108, :112, :]
####################################################
#shuffle dataset
fit_y = np.asarray(fit_y).astype('float32')
TRAIN_pickle_shuffle =np.array(TRAIN_pickle)
indices = list(range(len(TRAIN_pickle_shuffle)))
random.shuffle(indices)
TRAIN_npy_input = TRAIN_npy_input[indices,:,:,:,:]
TRAIN_pickle = TRAIN_pickle[indices,:]
fit_y = fit_y[indices]
sw = sw[indices]
###################################################
input_pickle = Input(shape=(TRAIN_pickle.shape[1],))
input_img = Input(shape=(TRAIN_npy_input.shape[1], 
                         TRAIN_npy_input.shape[2],
                         TRAIN_npy_input.shape[3],
                         TRAIN_npy_input.shape[4])
                  )

######   val_set  ######
with open('ADNI_TEST_set.pickle', 'rb') as f:
    ff_val = pickle.load(f)          

TEST_npy_input_val = np.load('ADNI_TEST_set.npy')

ttt_val = np.zeros([1,11],dtype="float32")
fir_val = np.concatenate((ttt_val,ff_val),axis = 0)
fir_val = fir_val[1:,:]

TEST_pickle_val = []    
for j_val in fir_val:
    c_val = [j_val[4],j_val[5]] # age and PTGENDER
    TEST_pickle_val.append(c_val)
    
TEST_pickle_val = pd.DataFrame(TEST_pickle_val)
TEST_pickle_val[1].replace("Female", 0).replace("Male", 1)
TEST_pickle_val = np.asarray(TEST_pickle_val).astype('float32')
#正規化
TEST_pickle_val[:,0] = (TEST_pickle_val[:,0]-50)/10   #年齡正規化

fir_val = pd.DataFrame(fir_val)
fit_y_val = []
for i in range(len(fir_val[7])):
    if (fir_val[7][i] >= 0.0)&(fir_val[7][i] <= 0.25):
        fit_y_val.append([1,0,0,0,0,0])
    elif (fir_val[7][i] > 0.25)&(fir_val[7][i] <= 2.75):
        fit_y_val.append([0,1,0,0,0,0])
    elif (fir_val[7][i] > 2.75)&(fir_val[7][i] <= 4.25):
        fit_y_val.append([0,0,1,0,0,0])
    elif (fir_val[7][i] > 4.25)&(fir_val[7][i] <= 9.25):
        fit_y_val.append([0,0,0,1,0,0])
    elif (fir_val[7][i] > 9.25)&(fir_val[7][i] <= 15.75):
        fit_y_val.append([0,0,0,0,1,0])
    elif fir_val[7][i] > 15.75:
        fit_y_val.append([0,0,0,0,0,1])
        
##############  class_weights  #######################
g = fit_y_val.count([1,0,0,0,0,0])
h = fit_y_val.count([0,1,0,0,0,0])
i = fit_y_val.count([0,0,1,0,0,0])
j = fit_y_val.count([0,0,0,1,0,0])
k = fit_y_val.count([0,0,0,0,1,0])
l = fit_y_val.count([0,0,0,0,0,1])
all_count_val = g+h+i+j+k+l
sw_dic_val = [all_count_val/g ,all_count_val/h ,all_count_val/i ,
           all_count_val/j ,all_count_val/k ,all_count_val/l]
del g ,h,i,j,k,l,all_count_val

sw_val = sw_dic_val[0]*np.ones(len(fit_y_val))
for ii in range(len(fit_y_val)):
    if fit_y_val[ii] == [1,0,0,0,0,0]:
        sw_val[ii] = sw_dic_val[0]
    elif fit_y_val[ii] == [0,1,0,0,0,0]:
        sw_val[ii] = sw_dic_val[1]
    elif fit_y_val[ii] == [0,0,1,0,0,0]:
        sw_val[ii] = sw_dic_val[2]
    elif fit_y_val[ii] == [0,0,0,1,0,0]:
        sw_val[ii] = sw_dic_val[3]
    elif fit_y_val[ii] == [0,0,0,0,1,0]:
        sw_val[ii] = sw_dic_val[4]
    elif fit_y_val[ii] == [0,0,0,0,0,1]:
        sw_val[ii] = sw_dic_val[5]

########################################################
TEST_npy_input_val = np.array(TEST_npy_input_val)
TEST_npy_input_val = np.concatenate((np.expand_dims(TEST_npy_input_val[:,:,:,:,0],axis=-1),np.expand_dims(TEST_npy_input_val[:,:,:,:,2],axis=-1)),axis=-1)
TEST_npy_input_val =TEST_npy_input_val[:, 7:135, 12:108, :112, :]
#########################################################
#shuffle dataset
fit_y_val = np.asarray(fit_y_val).astype('float32')
TEST_pickle_shuffle_val =np.array(TEST_pickle_val)
indices_val = list(range(len(TEST_pickle_shuffle_val)))
random.shuffle(indices_val)
TEST_npy_input_val = TEST_npy_input_val[indices_val,:,:,:,:]
TEST_pickle_val = TEST_pickle_val[indices_val,:]
fit_y_val = fit_y_val[indices_val,:]
sw_val = sw_val[indices_val]



In [ ]:
########################################################
def dice_coef(y_true, y_pred, smooth=1e-5):
    intersection = K.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (K.sum(y_true * y_true) + K.sum(y_pred * y_pred) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -K.log(dice_coef(y_true, y_pred))
######################    CBAM_block   #################
channel_axis = 1 if K.image_data_format() == "channel_first" else 4
def channel_attention(input_xs, reduction_ratio=0.125):
    # get channel
    channel = int(input_xs.shape[channel_axis])
    maxpool_channel = KL.GlobalMaxPooling3D()(input_xs)
    maxpool_channel = KL.Reshape((1,1,1,channel))(maxpool_channel)
    avgpool_channel = KL.GlobalAveragePooling3D()(input_xs)
    avgpool_channel = KL.Reshape((1,1,1,channel))(avgpool_channel) 
    Dense_One = KL.Dense(units=int(channel*reduction_ratio), activation='selu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    Dense_Two = KL.Dense(units=int(channel), activation='selu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    # max path
    mlp_1_max = Dense_One(maxpool_channel)
    mlp_2_max = Dense_Two(mlp_1_max)
    mlp_2_max = KL.Reshape(target_shape=(1,1,1,int(channel)))(mlp_2_max)
    # avg path
    mlp_1_avg = Dense_One(avgpool_channel)
    mlp_2_avg = Dense_Two(mlp_1_avg)
    mlp_2_avg = KL.Reshape(target_shape=(1,1,1,int(channel)))(mlp_2_avg)
    
    channel_attention_feature = KL.Add()([mlp_2_max, mlp_2_avg])
    channel_attention_feature = KL.Activation('sigmoid')(channel_attention_feature)
    return KL.Multiply()([channel_attention_feature, input_xs])

def spatial_attention(channel_refined_feature):
    maxpool_spatial = KL.Lambda(lambda x: K.max(x, axis=4, keepdims=True))(channel_refined_feature)
    avgpool_spatial = KL.Lambda(lambda x: K.mean(x, axis=4, keepdims=True))(channel_refined_feature)
    max_avg_pool_spatial = KL.Concatenate(axis=4)([maxpool_spatial, avgpool_spatial])
    return KL.Conv3D(filters=1, kernel_size=(3,3,3), padding="same", activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(max_avg_pool_spatial)

def cbam_module(input_xs, reduction_ratio=0.5):
    channel_refined_feature = channel_attention(input_xs, reduction_ratio=reduction_ratio)
    spatial_attention_feature = spatial_attention(channel_refined_feature)
    refined_feature = KL.Multiply()([channel_refined_feature,spatial_attention_feature])
    return KL.Add()([refined_feature, input_xs])
#########################################################
ipt1 = input_img

mod1_a = Conv3D(32, kernel_size=(3,3,3), padding='same', kernel_initializer='glorot_normal')(ipt1)
mod1_1 = BatchNormalization()(mod1_a)
mod1_1 = Concatenate(axis = -1)([mod1_1,ipt1])   #res
mod1_1 = Activation("selu")(mod1_1)
mod1_1 = cbam_module(mod1_1, reduction_ratio=0.125)
mod1_1 = MaxPooling3D(pool_size=(2,2,2), padding='same')(mod1_1)
del ipt1
mod1_2 = Conv3D(32, kernel_size=(3,3,3), padding='same',kernel_initializer='glorot_normal')(mod1_1)
mod1_2 = BatchNormalization()(mod1_2)
mod1_2 = Concatenate(axis = -1)([mod1_2,mod1_1])   #res
mod1_2 = Activation("selu")(mod1_2)
mod1_2 = MaxPooling3D(pool_size=(2,2,2), padding='same')(mod1_2)

mod1_3 = Conv3D(32, kernel_size=(3,3,3), padding='same',kernel_initializer='glorot_normal')(mod1_2)
mod1_3 = BatchNormalization()(mod1_3)
mod1_3 = Concatenate(axis = -1)([mod1_3,mod1_2])   #res
mod1_3 = Activation("selu")(mod1_3)
mod1_3 = cbam_module(mod1_3, reduction_ratio=0.125)
mod1_3 = MaxPooling3D(pool_size=(2,2,2), padding='same')(mod1_3)

mod1_4 = Conv3D(32, kernel_size=(3,3,3), padding='same',kernel_initializer='glorot_normal')(mod1_3)
mod1_4 = BatchNormalization()(mod1_4)
mod1_4 = Concatenate(axis = -1)([mod1_4,mod1_3])   #res
mod1_4 = Activation("selu")(mod1_4)
mod1_4 = MaxPooling3D(pool_size=(2,2,2), padding='same')(mod1_4)

mod1_5 = Conv3D(32, kernel_size=(3,3,3), padding='same',kernel_initializer='glorot_normal')(mod1_4)
mod1_5 = BatchNormalization()(mod1_5)
mod1_5 = Concatenate(axis = -1)([mod1_5,mod1_4])   #res
mod1_5 = Activation("selu")(mod1_5)
mod1_5 = cbam_module(mod1_5, reduction_ratio=0.125)
mod1_5 = MaxPooling3D(pool_size=(2,2,2), padding='same')(mod1_5)
mod1_5 = Dropout(0.4)(mod1_5)
opt = Conv3D(32,kernel_size=(1,1,1), padding='same',kernel_initializer='glorot_normal')(mod1_5)
opt = BatchNormalization()(opt)
opt = Activation("selu")(opt)
opt = Flatten()(opt)
opt = Dropout(0.4)(opt)
out_1 = Dense(32,activation=None,kernel_initializer='glorot_normal',kernel_regularizer= regularizers.l2(0.03))(opt)
out_1 = Activation('tanh')(out_1)
out_1 = Concatenate(axis=1)([out_1, input_pickle])
out_1 = Dense(32,activation=None,kernel_initializer='glorot_normal',kernel_regularizer= regularizers.l2(0.03))(out_1)
out_1 = Activation('tanh')(out_1)
out_1 = Dense(32,activation=None,kernel_initializer='glorot_normal',kernel_regularizer= regularizers.l2(0.03))(out_1)
out_1 = Activation('tanh')(out_1)
out_1 = Dense(32,activation=None,kernel_initializer='glorot_normal',kernel_regularizer= regularizers.l2(0.03))(out_1)
out_1 = Activation('tanh')(out_1)
out_1 = Dense(6,activation='softmax',kernel_initializer='glorot_normal',dtype='float32', kernel_regularizer= regularizers.l2(0.03))(out_1)

################  load_weights ################
model = Model(inputs =[input_img, input_pickle], outputs = out_1)
################################################
metrics = ['accuracy',
           dice_coef]
                                     
optimizers = keras.optimizers.Adam(lr = 5e-4)
model.compile(optimizer=optimizers, loss = dice_coef_loss , metrics = metrics) 

model.summary()

## callback
history = History()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-20, verbose=1)
model_checkpoint = ModelCheckpoint('6CDRclass_cut_conv5_32_4d32_D04_flatten_D04_mix_p5_cbam135-0125_dice_lr-4_l2003_b12-{epoch:05d}-{val_accuracy:.5f}' + '.h5', monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
csv_logger = CSVLogger(filename = '6CDRclass_cut_conv5_32_4d32_D04_flatten_D04_mix_p5_cbam135-0125_dice_lr-4_l2003_b12.csv' ,separator=',' ,append=False)

model.fit([TRAIN_npy_input, TRAIN_pickle],fit_y, sample_weight = sw, batch_size = 12 ,epochs = 200,verbose = 1,callbacks = [history,reduce_lr , model_checkpoint,csv_logger], validation_data =([TEST_npy_input_val, TEST_pickle_val],fit_y_val,sw_val))